--- Day 9: Rope Bridge ---

This rope bridge creaks as you walk along it. You aren't sure how old it is, or whether it can even support your weight.

It seems to support the Elves just fine, though. The bridge spans a gorge which was carved out by the massive river far below you.

You step carefully; as you do, the ropes stretch and twist. You decide to distract yourself by modeling rope physics; maybe you can even figure out where not to step.

Consider a rope with a knot at each end; these knots mark the head and the tail of the rope. If the head moves far enough away from the tail, the tail is pulled toward the head.

Due to nebulous reasoning involving Planck lengths, you should be able to model the positions of the knots on a two-dimensional grid. Then, by following a hypothetical series of motions (your puzzle input) for the head, you can determine how the tail will move.

Due to the aforementioned Planck lengths, the rope must be quite short; in fact, the head (H) and tail (T) must always be touching (diagonally adjacent and even overlapping both count as touching):
```
....
.TH.
....

....
.H..
..T.
....

...
.H. (H covers T)
...
```
If the head is ever two steps directly up, down, left, or right from the tail, the tail must also move one step in that direction so it remains close enough:
```
.....    .....    .....
.TH.. -> .T.H. -> ..TH.
.....    .....    .....

...    ...    ...
.T.    .T.    ...
.H. -> ... -> .T.
...    .H.    .H.
...    ...    ...
```
Otherwise, if the head and tail aren't touching and aren't in the same row or column, the tail always moves one step diagonally to keep up:
```
.....    .....    .....
.....    ..H..    ..H..
..H.. -> ..... -> ..T..
.T...    .T...    .....
.....    .....    .....

.....    .....    .....
.....    .....    .....
..H.. -> ...H. -> ..TH.
.T...    .T...    .....
.....    .....    .....
```
You just need to work out where the tail goes as the head follows a series of motions. Assume the head and the tail both start at the same position, overlapping.

For example:
```
R 4
U 4
L 3
D 1
R 4
D 1
L 5
R 2
```
This series of motions moves the head right four steps, then up four steps, then left three steps, then down one step, and so on. After each step, you'll need to update the position of the tail if the step means the head is no longer adjacent to the tail. Visually, these motions occur as follows (s marks the starting position as a reference point):
```
== Initial State ==

......
......
......
......
H.....  (H covers T, s)

== R 4 ==

......
......
......
......
TH....  (T covers s)

......
......
......
......
sTH...

......
......
......
......
s.TH..

......
......
......
......
s..TH.

== U 4 ==

......
......
......
....H.
s..T..

......
......
....H.
....T.
s.....

......
....H.
....T.
......
s.....

....H.
....T.
......
......
s.....

== L 3 ==

...H..
....T.
......
......
s.....

..HT..
......
......
......
s.....

.HT...
......
......
......
s.....

== D 1 ==

..T...
.H....
......
......
s.....

== R 4 ==

..T...
..H...
......
......
s.....

..T...
...H..
......
......
s.....

......
...TH.
......
......
s.....

......
....TH
......
......
s.....

== D 1 ==

......
....T.
.....H
......
s.....

== L 5 ==

......
....T.
....H.
......
s.....

......
....T.
...H..
......
s.....

......
......
..HT..
......
s.....

......
......
.HT...
......
s.....

......
......
HT....
......
s.....

== R 2 ==

......
......
.H....  (H covers T)
......
s.....

......
......
.TH...
......
s.....
```
After simulating the rope, you can count up all of the positions the tail visited at least once. In this diagram, s again marks the starting position (which the tail also visited) and # marks other positions the tail visited:
```
..##..
...##.
.####.
....#.
s###..
```
So, there are 13 positions the tail visited at least once.

Simulate your complete hypothetical series of motions. How many positions does the tail of the rope visit at least once?


In [23]:
// Data types and Functions
public enum Direction {
    Up,
    Down,
    Left,
    Right
}

Direction ParseDirection(string input) {
    return input switch {
        "U" => Direction.Up,
        "D" => Direction.Down,
        "L" => Direction.Left,
        "R" => Direction.Right,
        _ => throw new System.InvalidOperationException("Unknown direction"),
    };
}

Position Delta(Direction direction) {
    return direction switch {
        Direction.Up => new Position { X = 0, Y = -1 },
        Direction.Down => new Position { X = 0, Y = 1 },
        Direction.Left => new Position { X = -1, Y = 0 },
        Direction.Right => new Position { X = 1, Y = 0 },
    };
}

public record struct Move {
    public Direction Direction;
    public int Amount;
}

Move ParseMove(string line) {
    var parts = line.Split(" ");
    return new Move {
        Direction = ParseDirection(parts[0]),
        Amount = int.Parse(parts[1]),
    };
}

public record struct Position {
    public int X;
    public int Y;

    public static Position operator+(Position position, Position other) {
        return new Position {
            X = position.X + other.X,
            Y = position.Y + other.Y,
        };
    }
    
    public static Position operator-(Position position, Position other) {
        return new Position {
            X = position.X - other.X,
            Y = position.Y - other.Y,
        };
    }
}

public record struct State {
    public List<Position> Sections;
}

Position ApplySingleMoveToSection(Position newHead, Position currentTail, Direction direction) {
    var diff = newHead - currentTail;
    var newTailPosition = currentTail;
    if (Math.Abs(diff.X) > 1 && diff.Y == 0) {
        // Horizontal straight line
        newTailPosition = currentTail + new Position { X = Math.Sign(diff.X), Y = 0};
    } else if (Math.Abs(diff.Y) > 1 && diff.X == 0) {
        // Vertical straight line
        newTailPosition = currentTail + new Position { X = 0, Y = Math.Sign(diff.Y)};
    } else if ((Math.Abs(diff.X) + Math.Abs(diff.Y)) > 2) {
        // Diagonal
        newTailPosition = currentTail + new Position { X = Math.Sign(diff.X), Y = Math.Sign(diff.Y)};
    }
    return newTailPosition;
}

State ApplySingleMoveToState(State previous, Direction direction) {
    var newSections = previous.Sections.Skip(1)
        .Aggregate(
            new List<Position>{ previous.Sections[0] + Delta(direction) },
            (updatedSections, nextSectionPrevPosition) => updatedSections
                    .Append(ApplySingleMoveToSection(updatedSections.Last(), nextSectionPrevPosition, direction))
                    .ToList()
        );
    return new State {
        Sections = newSections
    };
}

List<State> ApplyMove(State state, Move move) {
    return Enumerable.Repeat(move.Direction, move.Amount)
        .Aggregate(
            new List<State> { state },
            (states, direction) => states.Append(ApplySingleMoveToState(states.Last(), direction)).ToList()
        );
}

In [24]:
// Read input
var input = System.IO.File.ReadAllLines("input.txt");
input.Take(10)

index,value
0,L 1
1,D 2
2,L 2
3,U 2
4,D 2
5,U 2
6,D 1
7,R 1
8,U 1
9,R 1


In [25]:
// Part 1
var state = new State {
    Sections = Enumerable.Repeat(new Position { X = 0, Y = 0 }, 2).ToList()
};
input
    .Select(ParseMove)
    .Aggregate(new List<State> { state }, (states, move) => states.Concat(ApplyMove(states.Last(), move)).ToList())
    .DistinctBy(state => state.Sections[1])
    .Count()

6357

In [26]:
// Part 2
var state = new State {
    Sections = Enumerable.Repeat(new Position { X = 0, Y = 0 }, 10).ToList()
};

input
    .Select(ParseMove)
    .Aggregate(new List<State> { state }, (states, move) => states.Concat(ApplyMove(states.Last(), move)).ToList())
    .DistinctBy(state => state.Sections[9])
    .Count()

2627